In [40]:
import torch
import chess
import chess.engine
from model import model
from data.vocab import BaseVocab
# from evaluation.evaluate import get_prediction
# from evaluation.evaluate import initialise_model
import evaluation.evaluate
from utils.utils import get_recent_ckpt
import IPython.display as vis

### Initialise the Model

In [41]:
# Initialise the model
gpt_model, stoi, itos = evaluation.evaluate.initialise_model('ckpts/epoch_1.pt')

AttributeError: module 'evaluation.evaluate' has no attribute 'initialise_model'

### Initialise the Chess Engine

In [7]:
engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")
# engine = chess.engine.SimpleEngine.popen_uci("/usr/local/bin/stockfish")

### Code for Bot vs. Human

In [8]:
def bot_vs_human(starting_pgn='', display_board=True):
    winner = None
    final_illegal = 0
    first_bad_move = -1

    # run inference loop
    game_str = starting_pgn
    bot_move = ''
    board = chess.Board()

    for move in game_str.split():
        board.push_san(move)

    illegal_moves = []
    if display_board:
        vis.display(board)
    bot_move_count = 0
    while True:
        user_move = input('Enter move: ')
        
        if user_move == "resign":
            break
        else:
            while True:
                try:
                    board.push_san(user_move)
                    game_str += user_move + ' '
                    break
                except ValueError:
                    display("MAKE A LEGAL MOVE")
                    user_move = input('Enter new move: ')
            
            
        if board.is_stalemate() or board.is_insufficient_material():
            winner = "DRAW"
            if display_board:
                vis.clear_output()
                vis.display(board)
                display("DRAW")
            break
        
        if board.is_checkmate():
            display("CHECKMATE, PLAYER WINS")
            break

        vis.clear_output()
        vis.display(board)

        # bot turn
        # handle cases where game str is larger than block size
        if len(game_str) >= 504:
            if display_board:
                vis.display("ALERT: Game string too long.  ChEPT resigns.")
                vis.clear_output()
                vis.display(board)
                display("CHECKMATE, CHePT RESIGNS")
            break
        bot_move = get_prediction(game_str, gpt_model, stoi, itos)
        bot_move_count += 1

        try:
            board.push_san(bot_move)
            illegal_moves.append(0)
        except ValueError:
            if first_bad_move == -1: first_bad_move = bot_move_count
            
                
            # try re-sampling 5 times
            success = False
            for i in range(15):
                bot_move = get_prediction(game_str, gpt_model, stoi, itos, sample=True)
                
                try:
                    board.push_san(bot_move)
                    success = True
                    break
                except:
                    pass

            if not success:
                if display_board:
                    vis.display("ALERT ALERT ALERT: Bot move was illegal.  Computer move substituted.")
                illegal_moves.append(1)
                final_illegal += 1
                bot_move = engine.play(board, chess.engine.Limit(time=0.1))
                bot_move_str = board.san(bot_move.move)
                board.push(bot_move.move)
                bot_move = bot_move_str
        
        if board.is_stalemate() or board.is_insufficient_material():
            winner = "DRAW"
            if display_board:
                vis.clear_output()
                vis.display(board)
                display("DRAW")
            break

        if board.is_checkmate():
            winner = "BOT"
            if display_board:
                vis.clear_output()
                vis.display(board)
                display("CHECKMATE, BOT WINS")
            break

        game_str = game_str + bot_move + ' '
        if display_board:
            vis.clear_output()
            vis.display(board)

    return (game_str, illegal_moves, first_bad_move, final_illegal, winner)

### Code for Bot vs. Stockfish

In [30]:
def check_end(just_moved, board, visualise):
    if visualise: vis.clear_output()
    if visualise: vis.display(board)
    if board.is_stalemate() or board.is_insufficient_material():
        winner = "DRAW"
        if visualise: display("DRAW")
        return (True, winner, 'stalemate')
    
    if board.is_checkmate():
        winner = just_moved
        if visualise: display(f"CHECKMATE, {winner} WINS")
        return (True, winner, 'checkmate')

    return (False, None, None)

def generate_legal_bot_move(board, game_str):
    illegal_attempts = 0
    for i in range(10):
        # don't bother sampling the first time
        sample = False if i == 0 else True
        bot_move = get_prediction(game_str.split(" "), gpt_model, stoi, itos, sample=sample)

        try:
            board.push_san(bot_move)
            return (bot_move, illegal_attempts)
        except ValueError:
            illegal_attempts += 1
    
    return None, illegal_attempts


def bot_vs_stockfish(starting_game_str='', visualise=False):
    game_str = starting_game_str

    board = chess.Board()

    for move in game_str.split():
        board.push_san(move)

    if visualise: vis.display(board)
    illegal_moves = []

    while True:
        # let the computer move
        comp_move = engine.play(board, chess.engine.Limit(time=0.05))
        game_str += board.san(comp_move.move)
        board.push(comp_move.move)

        over, winner, reason = check_end('STOCKFISH', board, visualise=visualise)
        if over:
            break

        # let our bot move
        bot_move, illegal_attempts = generate_legal_bot_move(board, game_str)
        illegal_moves.append(illegal_attempts)
        if bot_move == None:
            if visualise: display(f"CANNOT GENERATE LEGAL MOVE, STOCKFISH WINS")
            winner = 'STOCKFISH'
            reason = 'no-legal'
            break
        
        game_str += ' ' + bot_move + ' '
        over, winner, reason = check_end('BOT', board, visualise=visualise)
        if over:
            break

    return illegal_moves, winner, reason, game_str

In [32]:
bot_vs_stockfish('', visualise=False)

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10],
 'STOCKFISH',
 'no-legal',
 'e4 c5 Nc3 Nc6 Nf3 g6 d4 cxd4 Nxd4 Bg7 Be3 Nf6 Nxc6 bxc6 e5 Ng8 f4 Nh6 Be2 O-O Qd2 f6 exf6 exf6 O-O d5 Bc5 Re8 Rae1 Bf5 b4 Qd7 b5 cxb5 Bxb5 Qc7 Qxd5+ Kh8 Rxe8+ Rxe8 Bxe8 Qxc5+ Qxc5')